In [ ]:
import numpy as np
from scipy.ndimage import convolve

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

In [ ]:
# Replace '/path/to/chromedriver' with the actual path to the chromedriver executable
driver = webdriver.Chrome('chromedriver')

In [ ]:
# url, rows, cols, mine_num = 'https://minesweeperonline.com/#beginner', 9, 9, 10  # minesweeper beginner
# url, rows, cols, mine_num = 'https://minesweeperonline.com/#intermediate', 16, 16, 40  # minesweeper intermediate
url, rows, cols, mine_num = 'https://minesweeperonline.com/#expert', 16, 30, 99  # minesweeper intermediate
driver.get(url)

In [ ]:
class Minesweeper():
    def __init__(self, driver, rows, cols, mine_num):
        self.driver = driver
        self.actions = ActionChains(driver)
        self.rows = rows
        self.cols = cols

        
        
        self.end = 0
        self.win = 0
        self.lose = 0
        self.reset()
        
        return None
    
    def update_action(self, action):
        state = self.cell_state
        reward = 0
        self.action = np.zeros((self.rows * self.cols))
        self.action[action] = 1
        # self.action = np.reshape(action, (9,9))

        cell_ij = np.unravel_index(action, (self.rows, self.cols))
        
        
        reward_array_add = np.argmax(state, axis=-1,keepdims=True)[:,:,0]

        kernel = np.array([[1, 1, 1],
                    [1, 0, 1],
                    [1, 1, 1]])


        surrounding_sum = convolve(reward_array_add, kernel, mode='constant', cval=0.0)

        not_surrounded = (surrounding_sum % 9 == 0) * (surrounding_sum >= 27) * -10

        reward_array_add = np.where(reward_array_add == 9, 10, -10)

        # self.reward_array = reward_array_add + not_surrounded
        
        # self.reward_array = np.reshape(self.reward_array, (9,9))
        # print(state.shape, cell_ij)
        blocked = state[cell_ij[0], cell_ij[1],9] == 1

        element = self.driver.find_element_by_id(f"{cell_ij[0] + 1}_{cell_ij[1] + 1}")

        # Click on the element
        element.click()
        self.update()

        if self.end:
            if self.win:
                # self.reward_array[cell_ij[0], cell_ij[1]] += 25
                reward = 50
            else:
                # self.reward_array[cell_ij[0], cell_ij[1]] -= 15
                reward = -20
        elif not_surrounded[cell_ij[0], cell_ij[1]]:
            reward = -5
        else:
            reward = 5


        # elif blocked:
        #     if not_surrounded[cell_ij[0], cell_ij[1]] == 0:
        #         self.reward_array[cell_ij[0], cell_ij[1]] += 5
        #         reward = 5
        #     else:
        #         self.reward_array[cell_ij[0], cell_ij[1]] += 0
        #         reward = 0

            
        # else:
        #     self.reward_array[cell_ij[0], cell_ij[1]] = -20
        #     reward = -5

            
        return self.cell_state[:,:,:], reward, self.reward_array, self.end, self.win, self.lose
    
    def mark_mine(self, indicies):
        elements = []
        if len(indicies) > 0:
            if type(indicies[0]) is  np.ndarray:
                self.actions = ActionChains(driver)
                for index in indicies:
                    # print(f"{index}")

                    elements.append(self.driver.find_element_by_id(f"{index[0] + 1}_{index[1] + 1}"))

                    # driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('contextmenu', {'button': 2}))", element)
                    # Right-click on the element
                    self.actions.context_click(elements[-1])
                    # Execute JavaScript to click on the element
                    # driver.execute_script("arguments[0].click();", elements[-1])
                    # element_str_id = f"{index[0] + 1}_{index[1] + 1}"
                    
                    # driver.execute_script(f"document.getElementById('{element_str_id}').click();")
                self.actions.perform()
                del self.actions
                self.update
        return 1
        
    def uncover(self, indicies):
        elements = []
        self.actions = ActionChains(driver)
        for index in indicies:
            elements.append(self.driver.find_element_by_id(f"{index[0] + 1}_{index[1] + 1}"))

            # driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('contextmenu', {'button': 2}))", element)
            # Right-click on the element
            self.actions.click_and_hold(elements[-1])
            self.actions.context_click(elements[-1]).release()
        self.actions.perform()
        del self.actions
        self.update
        return 1

    def update(self):
        try:
            html_content = self.driver.page_source
        except Exception as e:
            print(e)
            quit()

        self.soup = BeautifulSoup(html_content, 'html.parser')
        self.game = self.soup.find("html").find("body").find("table").find("tbody").find("tr").find("td").find("div").find("div", id="center-column").find('div', id= "game-container").find('div', id = 'game')
        self.game1 = self.game.find_all(class_="square")
        self.end = 0 if self.game.find("div", class_="facesmile") else 1
        self.win = 1 if self.game.find("div", class_="facewin") else 0
        self.lose = 1 if self.game.find("div", class_="facedead") else 0
        self.cell_state[:,:,:] = 0
        # time0 = time.time()
        for i in range(1, self.rows + 1):
            for j in range(1, self.cols + 1):
                state = self.game1[(i - 1) * self.cols +j - 1]["class"][-1]
                match state:
                    case "blank":
                        self.cell_state[i-1,j-1,9] = 1
                    case value if value.startswith("open"):
                        self.cell_state[i-1,j-1, int(value[-1])] = 1
                    case value if value.startswith("bombflagged"):
                        self.cell_state[i-1,j-1,10] = 1
                    case _:
                        return 1

        # /html/body/table/tbody/tr/td/div/div[2]/div[1]/div[2]/div[1]
        # facewin
        # facesmile
        # facedead
        # print(time.time() - time0)



        return 1
    
    def reset(self):
        # Find the element by ID
        element = self.driver.find_element_by_id(f"face")  

        # Click on the element
        element.click()

        self.cell_state = np.zeros((self.rows,self.cols,11), dtype=np.float32)
        self.update()



        action = np.zeros(self.rows * self.cols, dtype=np.float32)
        action[np.random.randint(self.rows * self.cols)] = 1
        self.action = np.reshape(action, (self.rows,self.cols))
        cell_ij = np.unravel_index(np.argmax(self.action), self.action.shape)
        element = self.driver.find_element_by_id(f"{cell_ij[0] + 1}_{cell_ij[1] + 1}") 

        # Click on the element
        element.click()
        self.update()


        self.reward_array = np.argmax(self.cell_state, axis=-1,keepdims=True)

        self.reward_array = np.where(self.reward_array == 9, 2, -10)

        # self.update_action(action, self.cell_state)

        return 1


In [ ]:
gamer = Minesweeper(driver, rows, cols, mine_num)
reset = 0
uncovered_num = 0

In [ ]:
kernel = np.array([[1, 1, 1],
            [1, 0, 1],
            [1, 1, 1]])
not_done = True
while not_done:
    gamer.update()
    cell_states_0 = gamer.cell_state
    cell_states = np.argmax(cell_states_0[:,:,:-2],-1)
    cell_states_1 = np.argmax(cell_states_0[:,:],-1)
    uncovered_nums = np.argmax(cell_states_0[:,:,:-2],-1)
    # print(cell_states_0[:,:,10] == 1)
    mines = cell_states_1  == 10
    covered = cell_states_1  == 9
    uncovered = cell_states_1 <9
    print(uncovered_num, np.sum(uncovered))
    
    covered_surr = convolve(covered.astype(np.int8), kernel, mode='constant', cval=0.0).astype(np.bool_)
    if uncovered_num == np.sum(uncovered):
        reset = True
        # covered = (cell_states_0[:,:,9] == 1)
        # uncovered = np.ones_like(covered) ^ covered
        surrounding_sum = convolve(uncovered_nums.astype(np.int8), kernel, mode='constant', cval=0.0)
        surrounding_sum[uncovered] = 0
        covered_nums = surrounding_sum
        covered_border = (covered_nums > 0)
        covered_border[mines] = 0
        covered_border_order = convolve(covered_border.astype(np.int8), kernel, mode='constant', cval=0.0).astype(np.int8)
        covered_border_order[np.ones_like(covered_border) ^ covered_border] = 1


        # uncovered_border = convolve(covered_border.astype(np.int8), kernel, mode='constant', cval=0.0).astype(np.int8)
        # uncovered_border[np.ones_like(cell_states > 0) ^ uncovered_border] = 0
        uncovered_border = (cell_states > 0)
        uncovered_border = uncovered_border * convolve(covered_border, kernel, mode='constant', cval=0.0)



        uncovered_border_num_surr_covered = convolve(covered_border.astype(np.int8), kernel, mode='constant', cval=0.0).astype(np.int8)
        uncovered_border_num_surr_covered[np.ones_like(uncovered_border) ^ uncovered_border] = 1
        # not_surrounded = (surrounding_sum % 9 == 0) * (surrounding_sum >= 27) * -10
        surr_mines1 = convolve(mines, kernel, mode='constant', cval=0.0).astype(np.int8)
        probs = cell_states/uncovered_border_num_surr_covered
        probs[np.ones_like(uncovered_border) ^ uncovered_border] = 0
        highest_prob = np.max(probs)
        highest_prob_loc_uncovered = probs >= highest_prob
        highest_prob_loc_covered = convolve(highest_prob_loc_uncovered, kernel, mode='constant', cval=0.0).astype(np.int8)
        highest_prob_loc_covered = highest_prob_loc_covered * covered_border
        
        surr_mines = convolve(highest_prob_loc_covered, kernel, mode='constant', cval=0.0).astype(np.int8) * uncovered_border
        surr_mines = surr_mines * covered_surr
        
        # print("unmark")
        # highest_prob_loc_covered = highest_prob_loc_covered * (np.ones_like(mines) ^ mines)
        gamer.mark_mine(np.vstack(list(np.nonzero(highest_prob_loc_covered))).T)
        gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
        gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
        gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
        # highest_prob_loc_covered[np.ones_like(covered_border) ^ covered_border] = 0
    else:
        # print(mines.astype(np.int8), np.nonzero(mines.astype(np.int8)),np.vstack(list(np.nonzero(mines.astype(np.int8)))))
        # for something in np.vstack(list(np.nonzero(mines.astype(np.int8)))).T:
            # print("this", something)
        gamer.mark_mine(np.vstack(list(np.nonzero(mines.astype(np.int8)))).T)

    uncovered_num = np.sum(uncovered)
    not_done = not gamer.end

In [ ]:
# if reset:
#     for something in np.vstack(list(np.nonzero(mines))).T:
#         gamer.unmark_mine(something)
# for something in np.vstack(list(np.nonzero(highest_prob_loc_covered))).T:
# gamer.mark_mine(np.vstack(list(np.nonzero(highest_prob_loc_covered))).T)

# for something in np.vstack(list(np.nonzero(surr_mines))).T:
# gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
# gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
# gamer.uncover(np.vstack(list(np.nonzero(surr_mines))).T)
gamer.mark_mine(np.vstack(list(np.nonzero(surr_mines.astype(np.int8)))).T)

In [ ]:
# print(uncovered.astype(np.int8), "\n")
# print(covered.astype(np.int8), "\n")
# print(covered_border.astype(np.int8), "\n")
# print(surrounding_sum.astype(np.int8), "\n")
# covered_nums[uncovered] = 0
print('\n'.join(['   '.join([f'{item:.2f}' for item in row]) 
      for row in uncovered]), highest_prob)
# print(highest_prob_loc_uncovered.astype(np.int8))
# print(surr_mines1)
# print(probs)
# probs[surr_mines1.astype(np.bool_)] = 0
# probs[5,1] = 0
# print(uncovered_border.astype(np.int8), "\n")
# print(covered_border.astype(np.int8), "\n")
# print(uncovered_nums.astype(np.int8), "\n")
# print(covered_border_order.astype(np.int8), "\n")
print(probs >= highest_prob)